In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.ui import WebDriverWait
import threading
import time
import requests
import json
import pandas as pd
import cx_Oracle
from datetime import datetime

In [14]:
# driver = webdriver.Chrome(ChromeDriverManager().install())
# driver.get("https://www.naver.com")

스크래핑하는 함수 했음
데이터 프레임화 하는 함수 했음
데이터프레임을 오라클에 넣는 함수
실질적으로 돌아가는 함수

In [15]:
def bun_scraping_used_goods():
    url='https://m.bunjang.co.kr/search/products?category_id=700350&order=score&q='
    browser = webdriver.Chrome()
    browser.implicitly_wait(time_to_wait=10)
    browser.get(url)
    
    WebDriverWait(browser,10).until(expected_conditions.visibility_of_element_located((By.CLASS_NAME ,"app")))
    
    html = browser.page_source
    html_parser = BeautifulSoup(html,features="html.parser")
    
    list = html_parser.find_all(attrs={'alt':'상품 이미지'})
    
    img_list=[]
    for item in list:
        img_list.append(item['src'])
        
    name_list=[]
    price_list=[]
    link_list=[]
    for item in list:
        aTag1 = item.parent.parent.find_all('div','sc-fOICqy gQkWNx')
        aTag2 = item.parent.parent
        link_list.append("https://m.bunjang.co.kr{}".format(aTag2.attrs['href']))
        name_list.append(aTag1[0].find_all('div')[0].get_text())
        price_list.append(aTag1[0].find_all('div')[2].get_text())
        
    used_goods = pd.DataFrame({'제목' : name_list,
                          '가격': price_list,
                          '사진':img_list,
                          '링크' : link_list,
                          '사이트' : '번개장터'
                         })
    return used_goods

In [16]:
def bro_scraping_used_goods():
    url = 'https://wrightbrothers.kr/shop/bicycles/bicycles'
    browser = webdriver.Chrome()
    browser.get(url)
    
    WebDriverWait(browser,10).until(expected_conditions.visibility_of_element_located((By.CLASS_NAME ,"prod-title")))
    
    html = browser.page_source
    html_parser = BeautifulSoup(html,features="html.parser")
    
    bro_name=html_parser.select('div.prod-title > p.prod-model')

    bro_name_list=[]
    for item in bro_name:
        bro_name_list.append(item.get_text())
    
    bro_price=html_parser.select('span.price-num ')

    bro_price_list=[]
    for item in bro_price:
        bro_price_list.append(item.get_text())
        
    bro_link_img=html_parser.select('div.prod-card-item > a')
    bro_img_list=[]
    bro_link_list=[]
    for item in bro_link_img:
        bro_link_list.append("https://wrightbrothers.kr{}".format(item['href']))
        img=item['style']
        bro_img_list.append(img[img.find('"')+1:img.rfind('"')])
    
    used_goods = pd.DataFrame({'제목' : bro_name_list,
                          '가격': bro_price_list,
                          '사진':bro_img_list,
                          '링크' : bro_link_list,
                          '사이트' : '라이트'
                         })
    
    return used_goods

In [17]:
def concat_used_goods(df1,df2):
    return pd.concat([df1,df2])
    

In [5]:
# df=concat_used_goods(bun_scraping_used_goods(),bro_scraping_used_goods())

In [6]:
# df.head()

In [7]:
# df['가격']=df['가격'].str.replace(',','')
# df['가격']=df['가격'].str.replace('원','')
# df['가격']=df['가격'].astype('float64')

In [8]:
# df.columns

In [18]:
def preprocessed_df_to_oracle(df):

    con = cx_Oracle.connect('bike/12345@192.168.0.78:1521/xe')
    cur = con.cursor()
    sql_insert = '''
    MERGE INTO used_goods 
        USING dual
            ON (TITLE = :제목)
        WHEN NOT MATCHED THEN
            INSERT 
            VALUES (:제목2,:가격,:사진,:링크,:사이트) 
        WHEN MATCHED THEN
            UPDATE SET
                PRICE=:가격2
            '''
    
    
    
    for i in range(len(df)):
        제목 = df.iloc[i]['제목']
        제목2 = 제목
        가격 = df.iloc[i]['가격']
        가격2 = 가격
        사진 =df.iloc[i]['사진']
        링크 = df.iloc[i]['링크']
        사이트 = df.iloc[i]['사이트']

        cur.execute(sql_insert,(제목,제목2,가격,사진,링크,사이트,가격2))

    con.commit()
    cur.close()
    con.close()


In [19]:
def used_goods_info_collector():

    df = concat_used_goods(bun_scraping_used_goods(),bro_scraping_used_goods())
    df['가격']=df['가격'].str.replace(',','')
    df['가격']=df['가격'].str.replace('원','')
    df['가격']=df['가격'].astype('float64')
    preprocessed_df_to_oracle(df)

In [20]:
def used_goods_scheduler():
    print('중고거래 데이터 수집기 스케줄러 동작.\n')
    while True:
        used_goods_info_collector()
        print("수집완료.")
        time.sleep(6000) # 60분 주기로 데이터 수집

def print_main_menu():
    print('\n1. 중고거래 실시간 데이터 구축')
    print('2. 스케줄러 종료')
    print('* 엔터: 메뉴 업데이트\n')



In [ ]:
while True:
    print_main_menu()
    print('아래행에 메뉴입력: ')
    selection = input()
    
    if selection == '':
        continue
    else:
        menu_num = int(selection)
        
    if(menu_num == 1):
        t = threading.Thread(target=used_goods_info_collector, daemon=True)
        t.start()
    elif(menu_num == 2):
        break
    elif (menu_num == 0):
        continue


1. 중고거래 실시간 데이터 구축
2. 스케줄러 종료
* 엔터: 메뉴 업데이트

아래행에 메뉴입력: 
1

1. 중고거래 실시간 데이터 구축
2. 스케줄러 종료
* 엔터: 메뉴 업데이트

아래행에 메뉴입력: 


Exception in thread Thread-9:
Traceback (most recent call last):
  File "C:\Users\TJ\anaconda3\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "C:\Users\TJ\anaconda3\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\TJ\AppData\Local\Temp\ipykernel_10780\3254091390.py", line 7, in used_goods_info_collector
  File "C:\Users\TJ\AppData\Local\Temp\ipykernel_10780\427893896.py", line 28, in preprocessed_df_to_oracle
cx_Oracle.DatabaseError: ORA-00904: "TITLE": invalid identifier
